## Ablation: BLIP vs BLIP2

In [2]:
# Import libraries

import os
os.chdir("sandbox-DnD")

import clip
import utils

from sentence_transformers import SentenceTransformer

import pandas as pd

from evaluate import load

In [3]:
# Define variables

clip_name = 'ViT-B/16'
device = 'cuda'
batch_size = 200

In [4]:
# Define embedding models

mpnetmodel = SentenceTransformer('all-mpnet-base-v2')
clip_model, _ = clip.load(clip_name, device=device)
bertscore = load("bertscore")

### Load DnD Results

In [5]:
# Load CSVs

# DnD w/ BLIP
dnd_csv_layer1 = pd.read_csv("data/DnD_results/rn50_results/resnet50_imagenet_broden_layer1.csv")
dnd_csv_layer2 = pd.read_csv("data/DnD_results/rn50_results/resnet50_imagenet_broden_layer2.csv")
dnd_csv_layer3 = pd.read_csv("data/DnD_results/rn50_results/resnet50_imagenet_broden_layer3.csv")
dnd_csv_layer4 = pd.read_csv("data/DnD_results/rn50_results/resnet50_imagenet_broden_layer4.csv")

# DnD w/ BLIP2
dnd_blip2_csv_layer1 = pd.read_csv("data/DnD_results/BLIP2_data/BLIP2-Layer1.csv")
dnd_blip2_csv_layer2 = pd.read_csv("data/DnD_results/BLIP2_data/BLIP2-Layer2.csv")
dnd_blip2_csv_layer3 = pd.read_csv("data/DnD_results/BLIP2_data/BLIP2-Layer3.csv")
dnd_blip2_csv_layer4 = pd.read_csv("data/DnD_results/BLIP2_data/BLIP2-Layer4.csv")

In [6]:
# Get DnD results for each layer

# DnD w/ BLIP2
dnd_blip2_layer1 = []
dnd_blip2_layer2 = []
dnd_blip2_layer3 = []
dnd_blip2_layer4 = []
layer1_ids = []
layer2_ids = []
layer3_ids = []
layer4_ids = []
for i in range(len(dnd_blip2_csv_layer1)):
    dnd_blip2_layer1.append(dnd_blip2_csv_layer1["Label 1"][i])
    layer1_ids.append(dnd_blip2_csv_layer1["Neuron ID"][i])
for i in range(len(dnd_blip2_csv_layer2)):
    dnd_blip2_layer2.append(dnd_blip2_csv_layer2["Label 1"][i])
    layer2_ids.append(dnd_blip2_csv_layer2["Neuron ID"][i])
for i in range(len(dnd_blip2_csv_layer3)):
    dnd_blip2_layer3.append(dnd_blip2_csv_layer3["Label 1"][i])
    layer3_ids.append(dnd_blip2_csv_layer3["Neuron ID"][i])
for i in range(len(dnd_blip2_csv_layer4)):
    dnd_blip2_layer4.append(dnd_blip2_csv_layer4["Label 1"][i])
    layer4_ids.append(dnd_blip2_csv_layer4["Neuron ID"][i])
all_dnd_blip2 = dnd_blip2_layer1 + dnd_blip2_layer2 + dnd_blip2_layer3 + dnd_blip2_layer4

# DnD w/ BLIP
dnd_layer1 = []
dnd_layer2 = []
dnd_layer3 = []
dnd_layer4 = []
for i in range(len(dnd_csv_layer1)):
    if dnd_csv_layer1["Neuron ID"][i] in layer1_ids:
        dnd_layer1.append(dnd_csv_layer1["Label 1"][i])
for i in range(len(dnd_csv_layer2)):
    if dnd_csv_layer2["Neuron ID"][i] in layer2_ids:
        dnd_layer2.append(dnd_csv_layer2["Label 1"][i])
for i in range(len(dnd_csv_layer3)):
    if dnd_csv_layer3["Neuron ID"][i] in layer3_ids:
        dnd_layer3.append(dnd_csv_layer3["Label 1"][i])
for i in range(len(dnd_csv_layer4)):
    if dnd_csv_layer4["Neuron ID"][i] in layer4_ids:
        dnd_layer4.append(dnd_csv_layer4["Label 1"][i])
all_dnd = dnd_layer1 + dnd_layer2 + dnd_layer3 + dnd_layer4

### Find Cosine Similarity

In [7]:
# CLIP cosine similarities between DnD w/ BLIP labels and DnD w/ BLIP2 labels

clip_cos, _ = utils.get_cos_similarity(dnd_layer1, dnd_blip2_layer1, clip_model, mpnetmodel, device, batch_size)
print("Layer 1: " + str(clip_cos))
clip_cos, _ = utils.get_cos_similarity(dnd_layer2, dnd_blip2_layer2, clip_model, mpnetmodel, device, batch_size)
print("Layer 2: " + str(clip_cos))
clip_cos, _ = utils.get_cos_similarity(dnd_layer3, dnd_blip2_layer3, clip_model, mpnetmodel, device, batch_size)
print("Layer 3: " + str(clip_cos))
clip_cos, _ = utils.get_cos_similarity(dnd_layer4, dnd_blip2_layer4, clip_model, mpnetmodel, device, batch_size)
print("Layer 4: " + str(clip_cos))
clip_cos, _ = utils.get_cos_similarity(all_dnd, all_dnd_blip2, clip_model, mpnetmodel, device, batch_size)
print("All Layers: " + str(clip_cos))

Layer 1: 0.83935546875
Layer 2: 0.84375
Layer 3: 0.84228515625
Layer 4: 0.8681640625
All Layers: 0.84814453125


### Find BERT score

In [9]:
# BERT scores between DnD w/ BLIP labels and DnD w/ BLIP2 labels

bert_score = sum(bertscore.compute(predictions=dnd_layer1, references=dnd_blip2_layer1, lang="en")["f1"]) / len(bertscore.compute(predictions=dnd_layer1, references=dnd_blip2_layer1, lang="en")["f1"])
print("Layer 1: " + str(bert_score))
bert_score = sum(bertscore.compute(predictions=dnd_layer2, references=dnd_blip2_layer2, lang="en")["f1"]) / len(bertscore.compute(predictions=dnd_layer2, references=dnd_blip2_layer2, lang="en")["f1"])
print("Layer 2: " + str(bert_score))
bert_score = sum(bertscore.compute(predictions=dnd_layer3, references=dnd_blip2_layer3, lang="en")["f1"]) / len(bertscore.compute(predictions=dnd_layer3, references=dnd_blip2_layer3, lang="en")["f1"])
print("Layer 3: " + str(bert_score))
bert_score = sum(bertscore.compute(predictions=dnd_layer4, references=dnd_blip2_layer4, lang="en")["f1"]) / len(bertscore.compute(predictions=dnd_layer4, references=dnd_blip2_layer4, lang="en")["f1"])
print("Layer 4: " + str(bert_score))
bert_score = sum(bertscore.compute(predictions=all_dnd, references=all_dnd_blip2, lang="en")["f1"]) / len(bertscore.compute(predictions=all_dnd, references=all_dnd_blip2, lang="en")["f1"])
print("All Layers: " + str(bert_score))

Layer 1: 0.8826496315002441
Layer 2: 0.8799508821964264
Layer 3: 0.8939113056659699
Layer 4: 0.890931830406189
All Layers: 0.8868609139323235
